<a href="https://colab.research.google.com/github/i-rybachok/RybachokMachineLearning/blob/main/%D0%A0%D0%B8%D0%B1%D0%B0%D1%87%D0%BE%D0%BA_%D0%86_%D0%86_%D0%9C%D0%9D_%D0%A2%D0%B5%D0%BC%D0%B0_13_CNN_%D0%9A%D0%BB%D0%B0%D1%81_%D0%B4%D0%BE%D1%80_%D0%B7%D0%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
meowmeowmeowmeowmeow_gtsrb_german_traffic_sign_path = kagglehub.dataset_download('meowmeowmeowmeowmeow/gtsrb-german-traffic-sign')

print('Data source import complete.')


Рибачок Ірина, ФІТ 4-8

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from tensorflow.keras.layers import (Input, Rescaling, RandomFlip, RandomRotation, RandomContrast,
                                     Conv2D, BatchNormalization, MaxPool2D,
                                     GlobalAveragePooling2D, Dense, Dropout)

from tensorflow.keras.regularizers import l2


ВАРІАНТ 1

In [ ]:
# Шлях до директорії з тренувальними зображеннями
train_images_dir = '/kaggle/input/gtsrb-german-traffic-sign/Train'

# Списки для зображень і міток
images = []
labels = []

# Проходимо по кожній папці (класу)
for class_folder in os.listdir(train_images_dir):
    class_folder_path = os.path.join(train_images_dir, class_folder)

    # Перевіряємо, чи є це папка, і якщо так, то зберігаємо зображення
    if os.path.isdir(class_folder_path):
        for image_file in os.listdir(class_folder_path):
            # Створення шляху до зображення
            img_path = os.path.join(class_folder_path, image_file)
            # Завантаження зображення
            img = image.load_img(img_path, target_size=(32, 32))  # зміна розміру на 32x32
            img_array = image.img_to_array(img)

            # Додаємо зображення в список
            images.append(img_array)
            # Додаємо мітку класу (число)
            labels.append(int(class_folder))

# Перетворюємо списки у NumPy масиви
images = np.array(images)
labels = np.array(labels)

# Перевірка форми даних
print(f"Форма зображень: {images.shape}")
print(f"Мітки класів: {labels.shape}")

In [ ]:
# Поділ на тренувальні та тестові дані
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.2, random_state=42, stratify=labels
)

print(f"Розмір тренувальних даних: {X_train.shape}")
print(f"Розмір тестових даних: {X_test.shape}")

In [ ]:
# Перетворення міток на one-hot encoding
y_train = to_categorical(y_train, num_classes=43)
y_test = to_categorical(y_test, num_classes=43)

print(f"Форма y_train після перетворення: {y_train.shape}")  # Очікується (кількість_зразків, 43)
print(f"Форма y_test після перетворення: {y_test.shape}")    # Очікується (кількість_зразків, 43)

In [ ]:
height, width, channels = 32, 32, 3
NUM_CATEGORIES = 43  # Кількість класів для GTSRB

# Створення моделі
model = Sequential([
    Input(shape=(height, width, channels)),

    # Нормалізація + аугментації
    Rescaling(1./255),
    RandomFlip("horizontal"),
    RandomRotation(0.05),
    RandomContrast(0.1),

    # Блок 1
    Conv2D(32, (3,3), padding="same", activation="relu"),
    BatchNormalization(),
    Conv2D(32, (3,3), padding="same", activation="relu"),
    BatchNormalization(),
    MaxPool2D(),

    # Блок 2
    Conv2D(64, (3,3), padding="same", activation="relu"),
    BatchNormalization(),
    Conv2D(64, (3,3), padding="same", activation="relu"),
    BatchNormalization(),
    MaxPool2D(),

    # Блок 3
    Conv2D(128, (3,3), padding="same", activation="relu", kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    MaxPool2D(),

    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(128, activation="relu"),
    Dropout(0.3),
    Dense(NUM_CATEGORIES, activation="softmax")
])



In [ ]:
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import TopKCategoricalAccuracy

LOSS = CategoricalCrossentropy()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
    loss=LOSS,
    metrics=[
        "accuracy",
        TopKCategoricalAccuracy(k=3, name="top3")
    ]
)

cb = [
    tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=8, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1)
]

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),  # щоб працював val_accuracy і val_loss
    epochs=50,
    batch_size=32,
    shuffle=True,
    callbacks=cb,
    verbose=1
)

In [ ]:
training_accuracy = history.history["accuracy"]
training_loss = history.history["loss"]
validation_accuracy = history.history["val_accuracy"]
validation_loss = history.history["val_loss"]

In [ ]:
plt.figure(figsize=(10,5))

plt.subplot(1, 2, 1)
plt.plot(training_accuracy, color = "red", label = "Training Accuracy")
plt.plot(validation_accuracy, color = "cyan", label = "Validation Accuracy")
plt.legend()
plt.grid(True)
plt.title("Accuracy")
plt.xlabel("Number of Epochs")
plt.ylabel("Accuracy")

plt.subplot(1, 2, 2)
plt.plot(training_loss, color = "red", label = "Training Loss")
plt.plot(validation_loss, color = "cyan", label = "Validation Loss")
plt.grid(True)
plt.legend()
plt.title("Loss")
plt.xlabel("Number of epochs")
plt.ylabel("Loss")

plt.tight_layout()
plt.show()


In [ ]:
random_indices = np.random.choice(len(X_test), size=10, replace=False)

# Прогнозування класів для тестових даних
y_pred = model.predict(X_test)

# Виведення зображень та їхніх прогнозованих і справжніх класів
plt.figure(figsize=(15, 10))
for i, idx in enumerate(random_indices):
    plt.subplot(2, 5, i + 1)
    plt.imshow(X_test[idx])  # Показуємо зображення
    plt.axis('off')

    true_class = np.argmax(y_test[idx])  # Справжній клас
    predicted_class = np.argmax(y_pred[idx])  # Прогнозований клас

    plt.title(f"True: {true_class}, Pred: {predicted_class}")

plt.tight_layout()
plt.show()